# Tensor

tensor的接口有意设计成与Numpy类似

从接口的角度来讲，对tensor的操作可分为两类：

1. `torch.function`，如`torch.save`等。
2. 另一类是`tensor.function`，如`tensor.view`等。

为方便使用，对tensor的大部分操作同时支持这两类接口，在本书中不做具体区分，如`torch.sum (torch.sum(a, b))`与`tensor.sum (a.sum(b))`功能等价。

而从存储的角度来讲，对tensor的操作又可分为两类：

1. 不会修改自身的数据，如 `a.add(b)`， 加法的结果会返回一个新的tensor。
2. 会修改自身的数据，如 `a.add_(b)`， 加法的结果仍存储在a中，a被修改了。

函数名以`_`结尾的都是inplace方式, 即会修改调用者自己的数据，在实际应用中需加以区分。

## 创建Tensor

在PyTorch中新建tensor的方法有很多，具体如表3-1所示。

表3-1: 常见新建tensor的方法

|函数|功能|
|:---:|:---:|
|Tensor(\*sizes)|基础构造函数|
|tensor(data,)|类似np.array的构造函数|
|ones(\*sizes)|全1Tensor|
|zeros(\*sizes)|全0Tensor|
|eye(\*sizes)|对角线为1，其他为0|
|arange(s,e,step)|从s到e，步长为step|
|linspace(s,e,steps)|从s到e，均匀切分成steps份|
|rand/randn(\*sizes)|均匀/标准分布|
|normal(mean,std)/uniform(from,to)|正态分布/均匀分布|
|randperm(m)|随机排列|

这些创建方法都可以在创建的时候指定数据类型dtype和存放device(cpu/gpu).


其中使用`Tensor`函数新建tensor是最复杂多变的方式，它既可以接收一个list，并根据list的数据新建tensor，也能根据指定的形状新建tensor，还能传入其他的tensor，下面举几个例子。

In [75]:
import torch
import torch as t

In [76]:
print(torch.ones((2, 3)))
print(torch.ones(2, 3))
print(torch.tensor([1, 2, 3]))
print(torch.zeros(2, 3))
print(torch.eye(4, 3))
print(torch.arange(0, 10, 1))
print(torch.linspace(0, 10, 2))
print(torch.linspace(0, 10, 3))
print(torch.linspace(0, 10, 4))
print(torch.rand(3, 3))
print(torch.randn(3, 3))
print(torch.randperm(5))

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([1, 2, 3])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 0.]])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([ 0., 10.])
tensor([ 0.,  5., 10.])
tensor([ 0.0000,  3.3333,  6.6667, 10.0000])
tensor([[0.1536, 0.9095, 0.5796],
        [0.2755, 0.3723, 0.5135],
        [0.9135, 0.3705, 0.8773]])
tensor([[ 0.9900, -2.4872, -0.4560],
        [-0.1397, -0.4504, -0.7091],
        [ 0.3335, -0.4927, -0.0779]])
tensor([3, 4, 1, 0, 2])


In [77]:
# scalar: 没有维度的 tensor
print('scalar: ', torch.tensor(4325))

scalar:  tensor(4325)


In [78]:
# Tensor 和 tensor
# 以及不同的定义方式的区别
print(torch.Tensor(3, 4))
print(torch.Tensor([3, 4]))
print(torch.tensor([3, 4]))

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([3., 4.])
tensor([3, 4])


## 形状操作

通过`tensor.view`方法可以调整tensor的形状，但必须保证调整前后元素总数一致。`view`不会修改自身的数据，返回的新tensor与源tensor共享内存，也即更改其中的一个，另外一个也会跟着改变。在实际应用中可能经常需要添加或减少某一维度，这时候`squeeze`和`unsqueeze`两个函数就派上用场了。

In [79]:
a = torch.arange(0, 6)
a.view(2, 3)

tensor([[0, 1, 2],
        [3, 4, 5]])

In [80]:
a.view(-1, 3) # -1代表自动推断

tensor([[0, 1, 2],
        [3, 4, 5]])

In [81]:
b = a.view(2, 3)
b.shape, b.unsqueeze(1).shape, b.unsqueeze(1) # 第一维（从零开始，也就是第二维）增加一个维度

(torch.Size([2, 3]),
 torch.Size([2, 1, 3]),
 tensor([[[0, 1, 2]],
 
         [[3, 4, 5]]]))

In [82]:
b.shape, b.unsqueeze(-2).shape, b.unsqueeze(-2) # 倒数第二个维度 增加一个维度

(torch.Size([2, 3]),
 torch.Size([2, 1, 3]),
 tensor([[[0, 1, 2]],
 
         [[3, 4, 5]]]))

In [83]:
c = b.view(1, 1, 1, 2, 3)
c, c.shape

(tensor([[[[[0, 1, 2],
            [3, 4, 5]]]]]),
 torch.Size([1, 1, 1, 2, 3]))

In [84]:
c.shape, c.squeeze(0).shape, c.squeeze(0) # 第0维减少一个维度

(torch.Size([1, 1, 1, 2, 3]),
 torch.Size([1, 1, 2, 3]),
 tensor([[[[0, 1, 2],
           [3, 4, 5]]]]))

In [85]:
c.shape, c.squeeze().shape, c.squeeze() # 所有大小为1的维全部删掉

(torch.Size([1, 1, 1, 2, 3]),
 torch.Size([2, 3]),
 tensor([[0, 1, 2],
         [3, 4, 5]]))

In [86]:
a[1] = 100
b # 共享内存，a修改了b也修改了

tensor([[  0, 100,   2],
        [  3,   4,   5]])

In [87]:
b.resize_(1, 3) # resize_: 如果大小超过自动填充，不足自动删掉
b

tensor([[  0, 100,   2]])

In [88]:
b.resize_(2, 3) # 因为和a的地址是连续的会自动加数据

tensor([[  0, 100,   2],
        [  3,   4,   5]])

In [89]:
b.resize_(3, 3) # 多出的大小会分配新空间

tensor([[                0,               100,                 2],
        [                3,                 4,                 5],
        [32932988893659237, 29273792722501740, 31244220638625897]])

In [90]:
# UserWarning: non-inplace resize is deprecated
# 不能执行：b.resize(2, 3)

## 索引操作

In [91]:
a = torch.randn(3, 4)
a

tensor([[ 1.2133,  1.8208, -0.8014,  0.4624],
        [ 0.0899, -0.8073,  1.4253,  1.2491],
        [-0.0513,  1.0814, -2.0194, -0.9864]])

In [92]:
a[0]

tensor([ 1.2133,  1.8208, -0.8014,  0.4624])

In [93]:
a[:, 0]

tensor([ 1.2133,  0.0899, -0.0513])

In [94]:
a[0][2], a[0, 2]

(tensor(-0.8014), tensor(-0.8014))

In [95]:
a[0, -1]

tensor(0.4624)

In [96]:
a[:2]

tensor([[ 1.2133,  1.8208, -0.8014,  0.4624],
        [ 0.0899, -0.8073,  1.4253,  1.2491]])

In [97]:
a[:2, 0:2]

tensor([[ 1.2133,  1.8208],
        [ 0.0899, -0.8073]])

In [98]:
# 区别：形状不同，前者多一个维度
a[0:1, :2], a[0, :2]

(tensor([[1.2133, 1.8208]]), tensor([1.2133, 1.8208]))

In [99]:
# a[None] 等于多一个维度
a[None], a[None].shape

(tensor([[[ 1.2133,  1.8208, -0.8014,  0.4624],
          [ 0.0899, -0.8073,  1.4253,  1.2491],
          [-0.0513,  1.0814, -2.0194, -0.9864]]]),
 torch.Size([1, 3, 4]))

In [100]:
a[None], a[None,:,:] # 等价

(tensor([[[ 1.2133,  1.8208, -0.8014,  0.4624],
          [ 0.0899, -0.8073,  1.4253,  1.2491],
          [-0.0513,  1.0814, -2.0194, -0.9864]]]),
 tensor([[[ 1.2133,  1.8208, -0.8014,  0.4624],
          [ 0.0899, -0.8073,  1.4253,  1.2491],
          [-0.0513,  1.0814, -2.0194, -0.9864]]]))

In [101]:
a[:, None, :], a.unsqueeze(1) # 等价

(tensor([[[ 1.2133,  1.8208, -0.8014,  0.4624]],
 
         [[ 0.0899, -0.8073,  1.4253,  1.2491]],
 
         [[-0.0513,  1.0814, -2.0194, -0.9864]]]),
 tensor([[[ 1.2133,  1.8208, -0.8014,  0.4624]],
 
         [[ 0.0899, -0.8073,  1.4253,  1.2491]],
 
         [[-0.0513,  1.0814, -2.0194, -0.9864]]]))

In [102]:
a > 1 # 返回一个 ByteTensor

tensor([[ True,  True, False, False],
        [False, False,  True,  True],
        [False,  True, False, False]])

In [103]:
a[a > 1] # 等价于 a.masked_select(a > 1)
# 用 bool 的矩阵来进行批量选择 等于做了一个mask

tensor([1.2133, 1.8208, 1.4253, 1.2491, 1.0814])

In [104]:
a[torch.LongTensor([0, 1])] # 第0行和第1行

tensor([[ 1.2133,  1.8208, -0.8014,  0.4624],
        [ 0.0899, -0.8073,  1.4253,  1.2491]])

In [105]:
a[torch.tensor([0, 1])]

tensor([[ 1.2133,  1.8208, -0.8014,  0.4624],
        [ 0.0899, -0.8073,  1.4253,  1.2491]])

其它常用的选择函数如表3-2所示。

表3-2常用的选择函数

函数|功能|
:---:|:---:|
index_select(input, dim, index)|在指定维度dim上选取，比如选取某些行、某些列 (其他维度保持不变)
masked_select(input, mask)|例子如上，a[a>0]，使用ByteTensor进行选取
non_zero(input)|非0元素的下标
gather(input, dim, index)|根据index，在dim维度上选取数据，输出的size与index一样


`gather`是一个比较复杂的操作，对一个2维tensor，输出的每个元素如下：

```python
out[i][j] = input[index[i][j]][j]  # dim=0
out[i][j] = input[i][index[i][j]]  # dim=1
```
三维tensor的`gather`操作同理，下面举几个例子。

In [106]:
a.index_select(0, torch.tensor([1, 2]))

tensor([[ 0.0899, -0.8073,  1.4253,  1.2491],
        [-0.0513,  1.0814, -2.0194, -0.9864]])

In [107]:
a.index_select(1, torch.tensor([2, 3]))

tensor([[-0.8014,  0.4624],
        [ 1.4253,  1.2491],
        [-2.0194, -0.9864]])

In [108]:
a.nonzero(), a.unsqueeze(-1).nonzero() # 不是零的所有下标

(tensor([[0, 0],
         [0, 1],
         [0, 2],
         [0, 3],
         [1, 0],
         [1, 1],
         [1, 2],
         [1, 3],
         [2, 0],
         [2, 1],
         [2, 2],
         [2, 3]]),
 tensor([[0, 0, 0],
         [0, 1, 0],
         [0, 2, 0],
         [0, 3, 0],
         [1, 0, 0],
         [1, 1, 0],
         [1, 2, 0],
         [1, 3, 0],
         [2, 0, 0],
         [2, 1, 0],
         [2, 2, 0],
         [2, 3, 0]]))

In [109]:
a = torch.arange(0, 16).view(4, 4)
a

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [110]:
# gather(input, dim, index)
# input和index要维数相同
# out[i][j] = input[index[i][j]][j]  # dim=0
# out[i][j] = input[i][index[i][j]]  # dim=1
# 换句话说，output 生成 和 index 大小一样的数组，用index数组里的值来替换dim维的索引
# 选取对角线的元素
a.gather(0, torch.tensor([[0, 1, 2, 3]])), a.gather(0, torch.tensor([[1, 2, 3, 0], [3, 2, 1, 0]]))

(tensor([[ 0,  5, 10, 15]]),
 tensor([[ 4,  9, 14,  3],
         [12,  9,  6,  3]]))

In [111]:
a.gather(1, torch.tensor([[0, 1, 2, 3]])), a.gather(1, torch.tensor([[1, 2, 3, 0], [3, 2, 1, 0]]))

(tensor([[0, 1, 2, 3]]),
 tensor([[1, 2, 3, 0],
         [7, 6, 5, 4]]))

In [112]:
index = torch.LongTensor([[3, 2, 1, 0]]).t()
index, a.gather(1, index) # 选取反对角线的元素

(tensor([[3],
         [2],
         [1],
         [0]]),
 tensor([[ 3],
         [ 6],
         [ 9],
         [12]]))

In [113]:
# 选取反对角线上的元素，注意与上面的不同
index = torch.LongTensor([[3,2,1,0]])
a.gather(0, index)

tensor([[12,  9,  6,  3]])

In [114]:
# 选取两个对角线上的元素
index = torch.LongTensor([[0,1,2,3],[3,2,1,0]]).t()
b = a.gather(1, index)
b

tensor([[ 0,  3],
        [ 5,  6],
        [10,  9],
        [15, 12]])

与`gather`相对应的逆操作是`scatter_`，`gather`把数据从input中按index取出，而`scatter_`是把取出的数据再放回去。注意`scatter_`函数是inplace操作。

```python
out = input.gather(dim, index)
-->近似逆操作
out = Tensor()
out.scatter_(dim, index)
```

In [115]:
c = torch.zeros(4, 4)
c.scatter_(1, index, b.float())

tensor([[ 0.,  0.,  0.,  3.],
        [ 0.,  5.,  6.,  0.],
        [ 0.,  9., 10.,  0.],
        [12.,  0.,  0., 15.]])

对tensor的任何索引操作仍是一个tensor，想要获取标准的python对象数值，需要调用`tensor.item()`, 这个方法只对包含一个元素的tensor适用

In [116]:
a[0, 0] # 依旧是tensor

tensor(0)

In [117]:
a[0, 0].item() # python float

0

In [118]:
d = a[0:1, 0:1, None]
d, d.shape, d.item() # 因为只有一个元素，所以可以直接 item()

(tensor([[[0]]]), torch.Size([1, 1, 1]), 0)

## 高级索引
PyTorch在0.2版本中完善了索引操作，目前已经支持绝大多数numpy的高级索引[^10]。高级索引可以看成是普通索引操作的扩展，但是高级索引操作的结果一般不和原始的Tensor共享内存。 
[^10]: https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html#advanced-indexing

但是我懒得学所以直接咕了。

## Tensor类型

Tensor有不同的数据类型，如表3-3所示，每种类型分别对应有CPU和GPU版本(HalfTensor除外)。默认的tensor是FloatTensor，可通过`t.set_default_tensor_type` 来修改默认tensor类型(如果默认类型为GPU tensor，则所有操作都将在GPU上进行)。Tensor的类型对分析内存占用很有帮助。例如对于一个size为(1000, 1000, 1000)的FloatTensor，它有`1000*1000*1000=10^9`个元素，每个元素占32bit/8 = 4Byte内存，所以共占大约4GB内存/显存。HalfTensor是专门为GPU版本设计的，同样的元素个数，显存占用只有FloatTensor的一半，所以可以极大缓解GPU显存不足的问题，但由于HalfTensor所能表示的数值大小和精度有限[^2]，所以可能出现溢出等问题。

[^2]: https://stackoverflow.com/questions/872544/what-range-of-numbers-can-be-represented-in-a-16-32-and-64-bit-ieee-754-syste

表3-3: tensor数据类型

| Data type                | dtype                             | CPU tensor                                                   | GPU tensor                |
| ------------------------ | --------------------------------- | ------------------------------------------------------------ | ------------------------- |
| 32-bit floating point    | `torch.float32` or `torch.float`  | `torch.FloatTensor`                                          | `torch.cuda.FloatTensor`  |
| 64-bit floating point    | `torch.float64` or `torch.double` | `torch.DoubleTensor`                                         | `torch.cuda.DoubleTensor` |
| 16-bit floating point    | `torch.float16` or `torch.half`   | `torch.HalfTensor`                                           | `torch.cuda.HalfTensor`   |
| 8-bit integer (unsigned) | `torch.uint8`                     | [`torch.ByteTensor`](https://pytorch.org/docs/stable/tensors.html#torch.ByteTensor) | `torch.cuda.ByteTensor`   |
| 8-bit integer (signed)   | `torch.int8`                      | `torch.CharTensor`                                           | `torch.cuda.CharTensor`   |
| 16-bit integer (signed)  | `torch.int16` or `torch.short`    | `torch.ShortTensor`                                          | `torch.cuda.ShortTensor`  |
| 32-bit integer (signed)  | `torch.int32` or `torch.int`      | `torch.IntTensor`                                            | `torch.cuda.IntTensor`    |
| 64-bit integer (signed)  | `torch.int64` or `torch.long`     | `torch.LongTensor`                                           | `torch.cuda.LongTensor`   |

 

各数据类型之间可以互相转换，`type(new_type)`是通用的做法，同时还有`float`、`long`、`half`等快捷方法。CPU tensor与GPU tensor之间的互相转换通过`tensor.cuda`和`tensor.cpu`方法实现，此外还可以使用`tensor.to(device)`。Tensor还有一个`new`方法，用法与`t.Tensor`一样，会调用该tensor对应类型的构造函数，生成与当前tensor类型一致的tensor。`torch.*_like(tensora)` 可以生成和`tensora`拥有同样属性(类型，形状，cpu/gpu)的新tensor。 `tensor.new_*(new_shape)` 新建一个不同形状的tensor。

In [119]:
# 设置默认tensor，注意参数是字符串
torch.set_default_tensor_type('torch.DoubleTensor')

In [120]:
a = torch.Tensor(2,3)
a.dtype # 现在a是DoubleTensor,dtype是float64

torch.float64

In [121]:
# 恢复之前的默认设置
torch.set_default_tensor_type('torch.FloatTensor')

In [122]:
b = a.float()
b.dtype

torch.float32

In [123]:
c = a.type_as(b)
c

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [124]:
a.new(2, 3) # 等价于torch.DoubleTensor(2,3)，建议使用a.new_tensor

tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

In [125]:
torch.zeros_like(a) # 等价于t.zeros(a.shape,dtype=a.dtype,device=a.device)

tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

In [126]:
torch.zeros_like(a, dtype=torch.int16) #可以修改某些属性

tensor([[0, 0, 0],
        [0, 0, 0]], dtype=torch.int16)

In [127]:
torch.rand_like(a)

tensor([[0.1629, 0.3174, 0.6776],
        [0.1652, 0.3716, 0.9279]], dtype=torch.float64)

In [128]:
a.new_ones(4, 5, dtype=torch.int)

tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], dtype=torch.int32)

In [129]:
a.new_tensor([3, 4])

tensor([3., 4.], dtype=torch.float64)

## 逐元素操作

这部分操作会对tensor的每一个元素(point-wise，又名element-wise)进行操作，此类操作的输入与输出形状一致。常用的操作如表3-4所示。

表3-4: 常见的逐元素操作

|函数|功能|
|:--:|:--:|
|abs/sqrt/div/exp/fmod/log/pow..|绝对值/平方根/除法/指数/求余/求幂..|
|cos/sin/asin/atan2/cosh..|相关三角函数|
|ceil/round/floor/trunc| 上取整/四舍五入/下取整/只保留整数部分|
|clamp(input, min, max)|超过min和max部分截断|
|sigmod/tanh..|激活函数

对于很多操作，例如div、mul、pow、fmod等，PyTorch都实现了运算符重载，所以可以直接使用运算符。如`a ** 2` 等价于`torch.pow(a,2)`, `a * 2`等价于`torch.mul(a,2)`。

其中`clamp(x, min, max)`的输出满足以下公式：
$$
y_i =
\begin{cases}
min,  & \text{if  } x_i \lt min \\
x_i,  & \text{if  } min \le x_i \le max  \\
max,  & \text{if  } x_i \gt max\\
\end{cases}
$$
`clamp`常用在某些需要比较大小的地方，如取一个tensor的每个元素与另一个数的较大值。

In [130]:
a = torch.arange(0, 6).view(2, 3).float()
torch.cos(a)

tensor([[ 1.0000,  0.5403, -0.4161],
        [-0.9900, -0.6536,  0.2837]])

In [131]:
a % 3 # 等价于t.fmod(a, 3)

tensor([[0., 1., 2.],
        [0., 1., 2.]])

In [132]:
a ** 2 # 等价于t.pow(a, 2)

tensor([[ 0.,  1.,  4.],
        [ 9., 16., 25.]])

In [133]:
# 取a中的每一个元素与3相比较大的一个 (小于3的截断成3)
print(a)
torch.clamp(a, min=3)

tensor([[0., 1., 2.],
        [3., 4., 5.]])


tensor([[3., 3., 3.],
        [3., 4., 5.]])

In [134]:
b = a.sin_()
a

tensor([[ 0.0000,  0.8415,  0.9093],
        [ 0.1411, -0.7568, -0.9589]])

##  归并操作 

此类操作会使输出形状小于输入形状，并可以沿着某一维度进行指定操作。如加法`sum`，既可以计算整个tensor的和，也可以计算tensor中每一行或每一列的和。常用的归并操作如表3-5所示。

表3-5: 常用归并操作

|函数|功能|
|:---:|:---:|
|mean/sum/median/mode|均值/和/中位数/众数|
|norm/dist|范数/距离|
|std/var|标准差/方差|
|cumsum/cumprod|累加/累乘|

以上大多数函数都有一个参数**`dim`**，用来指定这些操作是在哪个维度上执行的。关于dim(对应于Numpy中的axis)的解释众说纷纭，这里提供一个简单的记忆方式：

假设输入的形状是(m, n, k)

- 如果指定dim=0，输出的形状就是(1, n, k)或者(n, k)
- 如果指定dim=1，输出的形状就是(m, 1, k)或者(m, k)
- 如果指定dim=2，输出的形状就是(m, n, 1)或者(m, n)

size中是否有"1"，取决于参数`keepdim`，`keepdim=True`会保留维度`1`。注意，以上只是经验总结，并非所有函数都符合这种形状变化方式，如`cumsum`。

In [135]:
b = torch.ones(2, 3)
b, b.sum(dim=0, keepdim=True)

(tensor([[1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[2., 2., 2.]]))

In [136]:
b.sum(dim=0, keepdim=False)

tensor([2., 2., 2.])

In [137]:
b.sum(dim=1)

tensor([3., 3.])

In [138]:
a = torch.arange(0, 6).view(2, 3)
print(a)
a.cumsum(dim=1) # 沿行累加

tensor([[0, 1, 2],
        [3, 4, 5]])


tensor([[ 0,  1,  3],
        [ 3,  7, 12]])

## 比较

比较函数中有一些是逐元素比较，操作类似于逐元素操作，还有一些则类似于归并操作。常用比较函数如表3-6所示。

表3-6: 常用比较函数

|函数|功能|
|:--:|:--:|
|gt/lt/ge/le/eq/ne|大于/小于/大于等于/小于等于/等于/不等|
|topk|最大的k个数|
|sort|排序|
|max/min|比较两个tensor最大最小值|

表中第一行的比较操作已经实现了运算符重载，因此可以使用`a>=b`、`a>b`、`a!=b`、`a==b`，其返回结果是一个`ByteTensor`，可用来选取元素。max/min这两个操作比较特殊，以max来说，它有以下三种使用情况：
- t.max(tensor)：返回tensor中最大的一个数
- t.max(tensor,dim)：指定维上最大的数，返回tensor和下标
- t.max(tensor1, tensor2): 比较两个tensor相比较大的元素

至于比较一个tensor和一个数，可以使用clamp函数。下面举例说明。

In [139]:
a = torch.linspace(0, 15, 6).view(2, 3)
a

tensor([[ 0.,  3.,  6.],
        [ 9., 12., 15.]])

In [140]:
b = t.linspace(15, 0, 6).view(2, 3)
b

tensor([[15., 12.,  9.],
        [ 6.,  3.,  0.]])

In [141]:
a > b

tensor([[False, False, False],
        [ True,  True,  True]])

In [142]:
a[a > b] # a中大于b的元素

tensor([ 9., 12., 15.])

In [143]:
t.max(a)

tensor(15.)

In [145]:
value, index = t.max(b, dim=1)
value, index
# 第一个返回值的15和6分别表示第0行和第1行最大的元素
# 第二个返回值的0和0表示上述最大的数是该行第0个元素

(tensor([15.,  6.]), tensor([0, 0]))

In [146]:
t.max(a, b)

tensor([[15., 12.,  9.],
        [ 9., 12., 15.]])

In [147]:
# 比较a和10较大的元素
t.clamp(a, min=10)

tensor([[10., 10., 10.],
        [10., 12., 15.]])

## 线性代数

PyTorch的线性函数主要封装了Blas和Lapack，其用法和接口都与之类似。常用的线性代数函数如表3-7所示。

表3-7: 常用的线性代数函数

|函数|功能|
|:---:|:---:|
|trace|对角线元素之和(矩阵的迹)|
|diag|对角线元素|
|triu/tril|矩阵的上三角/下三角，可指定偏移量|
|mm/bmm|矩阵乘法，batch的矩阵乘法|
|addmm/addbmm/addmv/addr/badbmm..|矩阵运算
|t|转置|
|dot/cross|内积/外积
|inverse|求逆矩阵
|svd|奇异值分解

具体使用说明请参见官方文档[^3]，需要注意的是，矩阵的转置会导致存储空间不连续，需调用它的`.contiguous`方法将其转为连续。
[^3]: http://pytorch.org/docs/torch.html#blas-and-lapack-operations

In [148]:
b = a.t()
b.is_contiguous()

False

In [151]:
b.contiguous().is_contiguous()

True

## Numpy